# ANÁLISIS DINÁMICO BÁSICO DE CARTERAS

# Análisis de Carteras: Efecto de la Evolución de Precios y el Rebalanceo

Hasta el momento, nuestros estudios de las diferentes estrategias de carteras se han centrado en la selección de activos y la asignación de pesos sin considerar la evolución temporal de los precios de los activos ni la posibilidad de realizar ajustes periódicos o rebalanceo. Estos factores pueden tener un impacto significativo en el rendimiento y el riesgo de las carteras.

En este notebook, exploraremos un análisis básico que incorpora tanto la dinámica de los precios de los activos como la estrategia de rebalanceo. Examinaremos cómo estos elementos afectan la evolución del valor de diferentes carteras a lo largo del tiempo, permitiéndonos obtener una comprensión más profunda de cada estrategia de inversión en un contexto más dinámico y realista. Además, consideraremos el impacto de las comisiones o gastos asociados al rebalanceo, lo que nos permite evaluar de manera más completa el rendimiento neto de las carteras.



In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import cvxpy as cp
from tqdm.auto import tqdm
import statsmodels.api as sm
import warnings
from scipy.linalg import sqrtm

plt.style.use('ggplot')
warnings.simplefilter(action='ignore', category=FutureWarning)


Defino las funciones que calculan los pesos de las distintas carteras

In [ ]:
def cartera_min_vol (ret):
    
    ''' Función que calcula la cartera de mínima varianza para un DataFrame de rendimientos
    ret: DataFrame de rendimientos
    Retorna pesos_ajustados: Array con los pesos de la cartera de mínima varianza'''
    
    if isinstance(ret, pd.DataFrame): # Verifico que el argumento sea un DataFrame
    
        num_act = ret.shape[1]
        matriz_cov = ret.cov().to_numpy()
        
            #Variables de decisión
        pesos = cp.Variable(num_act)
        
        #Restricciones
        constraints = [pesos >= 0,
                    cp.sum(pesos) == 1,
                    ]
        
        riesgo = cp.quad_form(pesos, matriz_cov) # Riesgo de la cartera
        objective = cp.Minimize(riesgo) # Minimizar la varianza

        #Problema y resuelvo
        prob = cp.Problem(objective, constraints)
        resultado = prob.solve()

        pesos_ajustados = np.array([np.round(x, 3) if x > 10**-4 else 0  for x in pesos.value]) #Pongo a cero los pesos menores a 10^-4 y redondeo a 3 decimales

        return pesos_ajustados
    
    else:
        raise ValueError('La función cartera_min_vol solo acepta un DataFrame como argumento') # Si el argumento no es un DataFrame, lanzo un error

In [ ]:
def cartera_max_sharpe(ret, ret_rf):
    
    ''' Función que calcula la cartera de máximo índice de Sharpe para un DataFrame de rendimientos
    ret: DataFrame de rendimientos
    ret_rf: Rendimiento del activo libre de riesgo
    Retorna pesos_ajustados: Array con los pesos de la cartera de máximo índice de Sharpe'''
    
    if isinstance(ret, pd.DataFrame):
    
        num_act = ret.shape[1]
        matriz_cov = ret.cov().to_numpy()
        retornos_esperados = ret.mean()

        # Variable de decisión (pesos del portafolio)
        x = cp.Variable(num_act)
        # Riesgo (desviación estándar) del portafolio
        riesgo = cp.quad_form(x, matriz_cov)

        #Cálculo de pi como retornos esperados menos la rantabilidad del activo libre de riesgo
        pi = np.array(retornos_esperados - ret_rf)

        #Restricciones
        constraints = [pi @ x ==1, # para que el numerador sea 1
                    x>=0]       # sin posiciones cortas

        objective = cp.Minimize(riesgo) # Minimizo el riesgo

        # Problema de optimización
        problema = cp.Problem(objective, constraints)        

        # Resolver el problema
        resultado  = problema.solve(solver=cp.ECOS)

        # Normalizo los pesos
        pesos = x.value
        pesos /= pesos.sum()

        pesos_ajustados = np.array([np.round(x, 3) if x > 10**-4 else 0  for x in pesos])

        return pesos_ajustados
    
    else:
        raise ValueError('La función cartera_max_sharpe solo acepta un DataFrame como argumento')

In [ ]:
def cartera_risk_parity (ret):
    
    ''' 
    Función que calcula la cartera de riesgo paridad para un DataFrame de rendimientos
    ret: DataFrame de rendimientos
    Retorna pesos_ajustados: Array con los pesos de la cartera de riesgo paridad'''
    
    if isinstance(ret, pd.DataFrame):
        
        num_act = ret.shape[1]
        matriz_cov = ret.cov().to_numpy()
        retornos_esperados = ret.mean().to_numpy()
        
        b = 1/num_act

        x = cp.Variable(num_act)
        gamma = cp.Variable(num_act, nonneg=True)
        psi = cp.Variable(nonneg=True)

        z = matriz_cov @ x

        obj = cp.pnorm(b**0.5 * psi - gamma, p=2)
        ret = retornos_esperados.T @ x

        constraints = [cp.sum(x) == 1,
                    x >= 0,
                    cp.SOC(psi, sqrtm(matriz_cov) @ x)]

        for i in range(num_act):
            constraints += [cp.SOC(x[i] + z[i],
                                cp.vstack([2*gamma[i], x[i] - z[i]]))
                            ]

        objective = cp.Minimize(obj * 1000)
        prob = cp.Problem(objective, constraints)
        prob.solve()

        pesos_ajustados = np.array([np.round(xi, 3) if xi > 10**-4 else 0 for xi in x.value])

        return pesos_ajustados 
    
    
    else:
        raise ValueError('La función cartera_risk_parity solo acepta un DataFrame como argumento')

In [32]:
start_date = '1995-01-01'
precios_indice = yf.download("SPY", start=start_date)[["Adj Close"]] # Precios ajustados al cierre

[*********************100%%**********************]  1 of 1 completed


In [33]:
filepath = 'https://raw.githubusercontent.com/alfonso-santos/microcredencial-carteras-python-2023/main/Tema_5_APT/data/sp500_tickers.csv'
tickers_sp500 = list(pd.read_csv(filepath))

precios = yf.download(tickers_sp500, start=start_date)['Adj Close']

precios_activos_sp500 = precios.copy()
precios_activos_sp500.dropna(axis=1, inplace=True)
#ret_activos_sp500 = np.log(precios_activos_sp500).diff().dropna()

[*********************100%%**********************]  503 of 503 completed

3 Failed downloads:
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 1995-01-01 -> 2024-04-18)')
['CDAY', 'BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')


Elegimos un universo de activos formados por num_act_max activos elegidos aleatoriamente para generar las carteras

In [34]:
num_act_max = 50

num_columnas = precios_activos_sp500.shape[1]

# Generar índices aleatorios para seleccionar num_act_max activos sin repetición

# Fijar la semilla del generador de números aleatorios
np.random.seed(42)  # Puedes usar cualquier número entero como semilla

indices_aleatorios = np.random.choice(num_columnas, size=50, replace=False)

# Seleccionar las columnas del array original usando los índices aleatorios
precios_activos_select = precios_activos_sp500.iloc[:, indices_aleatorios]



## Evaluación de la Evolución de las Carteras

Vamos a analizar la evolución de diferentes carteras mediante el rebalanceo periódico de las mismas. Este rebalanceo se realizará cada `rebalan_per`, que está medido en meses. Nuestro objetivo es entender cómo estas carteras cambian y se adaptan con el tiempo bajo diferentes condiciones de mercado, lo cual es crucial para optimizar las estrategias de inversión a largo plazo.


In [ ]:
#TODO

## Resumen del Análisis de Rebalanceo y Variabilidad en los Pesos de los Activos

Nuestro análisis previo se centró en observar cómo el valor de las carteras cambia a lo largo del tiempo, sin abordar las transacciones de rebalanceo necesarias para mantener las estrategias de asignación de activos. Estas transacciones son cruciales, ya que influyen significativamente en los costos operativos y el rendimiento total de la cartera.

Además, nos enfocaremos en la variabilidad de los pesos asignados a cada activo durante los rebalanceos. Para ello, calcularemos y analizaremos la diferencia entre el peso más alto y el más bajo registrado por cada activo a lo largo de todos los periodos de rebalanceo. Este análisis nos ayudará a entender la volatilidad en la asignación de los activos y evaluar cómo los ajustes frecuentes pueden afectar la estabilidad y eficiencia de nuestras estrategias de inversión.





In [2]:
#TODO